# K-NN Training Overview
This notebook implements a full supervised machine-learning pipeline for plant-health classification using k-Nearest Neighbors (k-NN).

In [ ]:
import joblib
import random
import numpy as np

from typing import List
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder

from core.preprocessing import load_and_preprocess_data
from core.outlier_detection import remove_outliers
from core.feature_selection import select_features
from core.classification import (
    classify_with_knn,
    classify_with_knn_without_hyperparameter,
)
from core.visualization import (
    visualize_ground_truth,
    visualize_knn_decision_boundary,
)


## Setting Random Seeds
Ensures that any randomness in preprocessing or clustering is reproducible.
This allows consistent results when rerunning the notebook.

In [ ]:
seed = 42
random.seed(seed)
np.random.seed(seed)

## Data Loading & Preprocessing
Here, we load the plant-health dataset and perform basic preprocessing steps required before clustering.
We:
- Reads the CSV file.
- Removes non-informative columns (`Timestamp`, `Plant_ID`).
- Splits data into features (`X`) and labels (`y`).
- Fills missing values (if any) using column means.
- Encodes the target variable using `LabelEncoder` (for visualization).
- Standardizes numerical features using `StandardScaler` and returns both raw and scaled versions.

In [ ]:
X_raw, X_scaled, y, label_encoder, scaler = load_and_preprocess_data(
    "../data/plant_health_data.csv"
)

## Outlier Removal
Identify and remove anomalous samples from a dataset using the Local Outlier Factor (LOF) algorithm. 
It takes a scaled feature matrix (`X_scaled`) and corresponding labels (`y_encoded`), 
detects outliers based on how isolated each sample is compared to its local neighborhood, 
and returns a cleaned dataset with outliers removed.

After detecting outliers: 
 - Filters out all samples labeled as outliers
 - Returns the cleaned feature matrix and labels
 - Visualizes inliers and outliers using the first two features

From the plot, the outliers (red Xs) appear randomly scattered away from high-density regions. This suggests random measurement errors or extreme anomalies 
as there is no visible structure or cluster pattern among the outliers.
- They do not group into a separate meaningful cluster.
- They do not represent a rare class.
- They are unlikely to contain useful information

Therefore, we decided to remove these found outliers.

In [ ]:
X_clean, y_clean = remove_outliers(X_scaled, y)

## Feature Selection
Perform feature selection using the `SelectKBest` with `Mutual Information` as the scoring metric. 
It evaluates how informative each feature is in predicting the target labels, 
selects the top k most relevant features, and returns:
 - `X_selected`: the transformed dataset containing only the selected features
 - `selected_features`: the names of the chosen features

This process helps reduce dimensionality, remove irrelevant inputs, 
and improve the performance and interpretability of ML models. 

Feature selection is a crucial step as without performing feature selection (see: `train_without_feature_selection.ipynb`), the model performance significantly decreases approximately `10%`.

In [ ]:
X_selected, selected_features = select_features(X_clean, y_clean, X_raw.columns, k=8)

## Visualization
Two visualizations are provided:
- Ground truth scatter of selected features
- k-NN decision boundary plot (using 2D PCA) to understand classifier behaviour

In [ ]:
selected_scaler = StandardScaler()
X_selected_scaled = selected_scaler.fit_transform(X_selected)
visualize_ground_truth(X_selected_scaled, y_clean, label_encoder)
visualize_knn_decision_boundary(X_selected_scaled, y_clean, n_neighbors=5)

## Classification
Here we train `K-NN` classifier with hyperparameter tuning where
we search for optimal `k`.

With hyperparameter tuning, the `K-NN` classifier achieves approximately `78%` accuracy, compared to only `70%` accuracy without tuning (using `classify_with_knn_without_hyperparameter`). This shows that hyperparameter optimization provides a substantial performance gain for `K-NN`. 

Additionally, the poorest performance occurs when training `K-NN` without hyperparameter tuning and without feature selection, yielding only about `63%` accuracy, barely above a coin flip.

## ROC Curve
From the ROC Curve, the classifier distinguishes `Healthy` and `High Stress` extremely well, but is less confident when detecting `Moderate Stress`.

In [ ]:
# Train classifier with hyperparameter
print("Running KNN with hyperparameter tuning...")
knn = classify_with_knn(X_selected_scaled, y_clean, label_encoder)

## Saving Model Artifacts
The following model components are saved for later inference (`./inference.sh`):
- Trained k-NN model
- Label encoder
- Scaler for selected features
- List of selected feature names

In [ ]:
def save_model(
    knn: KNeighborsClassifier,
    label_encoder: LabelEncoder,
    scaler: StandardScaler,
    selected_features: List[str],
) -> None:
    path = './model'
    joblib.dump(knn, f"{path}/knn_model.pkl")
    joblib.dump(label_encoder, f"{path}/label_encoder.pkl")
    joblib.dump(scaler, f"{path}/scaler.pkl")
    joblib.dump(selected_features, f"{path}/selected_features.pkl")

    print("Model and label encoder saved.")
    
save_model(
    knn=knn, 
    label_encoder=label_encoder, 
    scaler=selected_scaler, 
    selected_features=selected_features,
)